# building individual a11yhood pages from scraped items

use dataframe techniques to create static site contents

In [143]:
from tonyfast.tonyfast.xxiv.schema_frame import tag, klass
import bs4, ipywidgets, uuid

load the a11yhood dataset

In [145]:
df = read_json("research/at.json.gz")

compose the components that we'll use for our site content.

In [176]:
header = tag.header(tag.a(tag.img(src="https://a11yhood.org/images/ahood-200.jpg"), **klass["brand"]))

In [188]:
sections = (
    df[["name"]].html.tag("h1").join(
        df.tags.dropna().apply(Series).html.tag("li").html.group("ul", **klass["tags"]).rename("tags")
    )
    # .join(df.assign(text="visit source").text.html.tag("a", href=df.index.to_series()))
    .join(
        df.assign(h2="description").h2.html.tag("h2")
    ).join(
        df.description[df.description.fillna("").astype(bool)].apply(
            compose_left(shell.tangle.parser.parser.render, partial(bs4.BeautifulSoup, features="lxml"))
        ).html.tag("p")
    ).join(
        df[["license"]].dropna().assign(h2="license").h2.html.tag("h2").to_frame("license_title").join(
            df.dropna().license.html.tag("p")
        )
    ).join(
        df.assign(style="""ul.tags {
            display: flex; flex-direction: row; list-style: none; li {margin: 10px;}
        }""")["style"].html.tag("style")
    ).html.group("section")
)

In [214]:
pages = df.assign(
    header=[bs4.BeautifulSoup(str(header), features="lxml").header for _ in range(len(df))]
)[["header"]].join(sections.rename('section')).html.group("section")

view intermediate html representations

In [215]:
@ipywidgets.interact
def show_pages(i = (0, len(pages))):
    return pipe(pages.iloc[i], HTML)

interactive(children=(IntSlider(value=602, description='i', max=1204), Output()), _dom_classes=('widget-intera…

## export the results

In [216]:
import nbformat, nbconvert
exporter = nbconvert.get_exporter("html")()

In [217]:
targets = pages.head(20).apply(
    lambda x: nbformat.v4.new_notebook(cells=[
        nbformat.v4.new_markdown_cell(str(x))
    ])
)

targets = targets.to_frame("data").combine_first(
    targets.apply(compose_left(
        nbformat.from_dict, exporter.from_notebook_node, first
    )).to_frame("html")
)

targets = targets.join(df.index.to_series().str.rpartition("/")[2].rename("target"))
targets = targets.set_index("target")
targets 

,data,html
target,,
ai-collection,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
OptiKey,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
awesome-a11y,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
dicio-android,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
cboard,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
whisper_dictation,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
voqal,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
DReyeVR,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."
righthand,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."


In [220]:
target = Path("site", "a11yhood", "items")

for path, row in targets.iterrows():
    target.mkdir(exist_ok=True, parents=True)
    (target / path).with_suffix(".html").write_text(str(row["html"]))

In [221]:
targeted = Index(target.glob("*.html")).rename("static").to_frame()[[]]

In [222]:
@ipywidgets.interact
def show_iframes(i: (0, len(targeted))):
    return IFrame(targeted.index[i], "100%", 600)

interactive(children=(IntSlider(value=10, description='i', max=20), Output()), _dom_classes=('widget-interact'…